In [1]:
# python env req:
# conda install -c conda-forge rubin-scheduler
# conda install -c lsst-efd-client
# conda install -c panel
# conda install -c holoviews
# pip install -e . ts_fbs_utils --no-deps (# git clone ts_fbs_utils)
# git clone ts_config_ocs

import os
import warnings
import copy
import pickle
import json
import numpy as np
import healpy as hp
import matplotlib.pylab as plt

import pandas as pd
import hvplot.pandas
import panel as pn
import hvplot

import datetime
from astropy.time import Time, TimeDelta
import astropy.units as u

from lsst_efd_client import EfdClient

from rubin_scheduler.scheduler import sim_runner
from rubin_scheduler.utils import ra_dec2_hpid
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from rubin_scheduler.site_models import Almanac, SeeingData
from rubin_scheduler.utils import Site
from rubin_scheduler.scheduler.features import Conditions
from rubin_scheduler.scheduler.utils import SchemaConverter

# Import the actually run config script from ts_config_ocs
# (run in directory containing a copy of the script)
from fbs_config_image_photocal_survey import get_scheduler

In [2]:
prenight=True

if prenight:

    pn.extension("terminal")
    
    #pn.extension()
    warnings.filterwarnings(
        "ignore",
        module="astropy.time",
        message="Numerical value without unit or explicit format passed to TimeDelta, assuming days",
    )
    warnings.filterwarnings(
        "ignore",
        module="pandas",
        message="In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.",
    )
    warnings.filterwarnings(
        "ignore",
        module="healpy",
        message="divide by zero encountered in divide",
    )
    warnings.filterwarnings(
        "ignore",
        module="healpy",
        message="invalid value encountered in multiply",
    )
    warnings.filterwarnings(
        "ignore",
        module="holoviews",
        message="Discarding nonzero nanoseconds in conversion.",
    )
    warnings.filterwarnings(
        "ignore",
        module="rubin_scheduler",
        message="invalid value encountered in arcsin",
    )
    warnings.filterwarnings(
        "ignore",
        module="rubin_sim",
        message="All-NaN slice encountered",
    )

In [12]:
# What night do you want? 
DAYOBS = '2024-02-06'
site = Site('LSST')
almanac = Almanac()
night_events = almanac.get_sunset_info(evening_date=DAYOBS, longitude=site.longitude_rad)
sunset = Time(night_events['sunset'], format='mjd', scale='utc')
sunrise = Time(night_events['sunrise'], format='mjd', scale='utc')
survey_length = sunrise.mjd - sunset.mjd
sunset.iso, sunrise.iso, survey_length

('2024-02-06 23:34:20.545', '2024-02-07 10:20:05.949', 0.44844217831268907)

In [13]:
# Set up simulation with scheduler
# Keep this cell together (so regenerate scheduler and observatory if rerun)

rewards = False 

nside, scheduler=get_scheduler()
scheduler.keep_rewards=rewards

observatory = ModelObservatory(nside=nside, mjd_start=sunset.mjd, ideal_conditions=True)
observatory.seeing_data = SeeingData(start_time)
# Disable shutter stall build into sim's model observatory 
observatory.observatory.setup_camera(shutter_2motion_min_time=0.0)

vals = sim_runner(
    observatory,
    scheduler,
    survey_length=survey_length,
    record_rewards=rewards,
    verbose=True,
)

/Users/lynnej/miniconda3/envs/ts-support/lib/python3.11/site-packages/rubin_scheduler/skybrightness_pre/sky_model_pre.py:280: UserWarning: Requested MJD between sunrise and sunset, returning closest maps
  warnings.warn("Requested MJD between sunrise and sunset, returning closest maps")


progress = 9.47%

/Users/lynnej/miniconda3/envs/ts-support/lib/python3.11/site-packages/rubin_scheduler/scheduler/schedulers/core_scheduler.py:280: RuntimeWarning: All-NaN axis encountered
  rewards[i] = np.nanmax(survey.calc_reward_function(self.conditions))
Failed to fill queue at time [60348.06567239]
/Users/lynnej/miniconda3/envs/ts-support/lib/python3.11/site-packages/rubin_scheduler/scheduler/sim_runner.py:96: UserWarning: No observation. Step into the future and trying again.
  warnings.warn("No observation. Step into the future and trying again.")


progress = 28.79%

Failed to fill queue at time [60348.1716685]


...

progress = 68.79%

Failed to fill queue at time [60348.33167696]
Failed to fill queue at time [60348.34816775]
Failed to fill queue at time [60348.36570671]


progress = 80.06%

Failed to fill queue at time [60348.3822331]
Failed to fill queue at time [60348.39264977]
Failed to fill queue at time [60348.40306643]
Failed to fill queue at time [60348.4134831]
Failed to fill queue at time [60348.42389977]
Failed to fill queue at time [60348.43431643]
Failed to fill queue at time [60348.4447331]
Failed to fill queue at time [60348.45514977]
Failed to fill queue at time [60348.46556643]
Failed to fill queue at time [60348.4759831]
Failed to fill queue at time [60348.48639977]
Failed to fill queue at time [60348.49681643]
Failed to fill queue at time [60348.5072331]
Failed to fill queue at time [60348.51764977]
Failed to fill queue at time [60348.52806643]
Failed to fill queue at time [60348.5384831]
Failed to fill queue at time [60348.54889977]
Failed to fill queue at time [60348.55931643]
Failed to fill queue at time [60348.5697331]
Failed to fill queue at time [60348.58014977]
Failed to fill queue at time [60348.59056643]
Failed to fill queue at time [60348.60098

progress = 224.21%Skipped 70 observations
Flushed 0 observations from queue for being stale
Completed 70 observations
ran in 0 min = 0.0 hours


In [14]:
observatory = vals[0]
scheduler = vals[1]
observations = vals[2]
if len(vals) == 5:
    rewards = vals[3]
    obs_rewards = vals[4]

In [15]:
schema_converter = SchemaConverter()
visits = schema_converter.obs2opsim(observations)

# Add "start_date" and "fix" note (remove _expnum)
visits["start_date"] = pd.to_datetime(
    visits["observationStartMJD"] + 2400000.5, origin="julian", unit="D", utc=True
)
def demangle_note(x):
    if "IM" in x.note:
        x.note = x.note.split(":")[1].split("_")[0]
    return x
visits = visits.apply(demangle_note, axis=1)

visits.describe().T

,count,mean,std,min,25%,50%,75%,max
observationId,70.0,34.500000,20.351085,0.000000,17.250000,34.500000,51.750000,69.000000
fieldRA,70.0,126.285101,57.395969,7.825000,86.500000,122.513981,129.466667,307.075000
fieldDec,70.0,-37.588764,17.180274,-88.483299,-36.174781,-32.306444,-26.255000,-24.146674
observationStartMJD,70.0,60348.172252,0.139575,60348.023245,60348.090660,60348.124891,60348.232288,60349.028168
flush_by_mjd,70.0,55175.480847,17015.951518,0.000000,60348.107072,60348.134388,60348.242158,60348.403230
visitExposureTime,70.0,279.428571,157.304152,30.000000,120.000000,390.000000,420.000000,420.000000
rotSkyPos,70.0,169.821971,86.837417,50.542523,80.099368,174.790820,253.368706,349.607303
rotSkyPos_desired,70.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
numExposures,70.0,1.257143,0.845889,1.000000,1.000000,1.000000,1.000000,4.000000
airmass,70.0,1.307912,0.389773,1.002965,1.043089,1.081942,1.484279,2.236222


In [18]:
# Plot airmass of simulated targets
visits.hvplot(
    by=["note"],
    kind="scatter",
    x="start_date",
    ylim=(2.3, 0.9),
    y=["airmass"],
)

:NdOverlay   [note]
   :Scatter   [start_date]   (airmass)

In [8]:
# Fetch info from EFD - check on configuration

good_locs = {"USDF" : "usdf", "SUMMIT": "summit"}

loc = "usdf"
efd_client = EfdClient(f"{loc}_efd")

topic = "lsst.sal.Scheduler.logevent_configurationApplied"
fields = ['configurations', 'otherInfo', 'schemaVersion', 'url']
dd = await efd_client.select_time_series(topic, fields, sunset, sunrise, index=2)
dd

,configurations,otherInfo,schemaVersion,url
2024-02-07 00:35:24.768939+00:00,"_init.yaml,_summit.yaml,auxtel_fbs_image_photo...",,v6,file:///net/obs-env/auto_base_packages/ts_conf...
2024-02-07 01:52:54.940545+00:00,"_init.yaml,_summit.yaml,auxtel_fbs_image_photo...",,v6,file:///net/obs-env/auto_base_packages/ts_conf...


In [9]:
# Fetch requested targets
topic = 'lsst.sal.Scheduler.logevent_target'
fields = ['ra', 'decl', 'skyAngle', 'note', 'slewTime', 'airmass', 
 'exposureTimes0',
 'exposureTimes1',
 'exposureTimes2',
 'exposureTimes3',
 'exposureTimes4',
 'exposureTimes5',
 'exposureTimes6',
 'exposureTimes7',
 'exposureTimes8',
 'exposureTimes9',
 'filter',
 'skyBrightness',]
targets = await efd_client.select_time_series(topic, fields, sunset, sunrise, index=2)
len(targets)

68

In [10]:
visits_req = copy.deepcopy(targets)
visits_req["start_date"] = targets.index
# Drop visits that are recorded but are not real (?)
visits_req = visits_ac.query("note != 'Target'")
# mangle the note again 
visits_req = visits_ac.apply(demangle_note, axis=1)

visits_req.describe().T

,count,mean,std,min,25%,50%,75%,max
ra,57.0,130.815674,45.131684,86.500000,122.456814,122.548397,129.466667,307.075000
decl,57.0,-34.864103,13.303925,-87.472194,-36.166185,-36.126740,-26.255000,-22.622563
skyAngle,57.0,31.715184,69.371293,0.000000,0.000000,16.448826,23.330080,326.508914
slewTime,57.0,24.427214,30.508673,2.000000,3.128132,3.286280,40.500260,120.000000
airmass,57.0,1.244521,0.354899,1.003970,1.015801,1.044329,1.365496,2.206427
exposureTimes0,57.0,256.315789,158.916606,7.500000,120.000000,120.000000,420.000000,420.000000
exposureTimes1,57.0,0.526316,1.932848,0.000000,0.000000,0.000000,0.000000,7.500000
exposureTimes2,57.0,0.526316,1.932848,0.000000,0.000000,0.000000,0.000000,7.500000
exposureTimes3,57.0,0.526316,1.932848,0.000000,0.000000,0.000000,0.000000,7.500000
exposureTimes4,57.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [17]:
# Observations are the actually acquired observations .. might be better to get this from butler 

topic = 'lsst.sal.Scheduler.logevent_observation'
fields = ['additionalInformation',
           'ra',
 'decl',
 'exptime',
 'filter',
 'mjd',
 'nexp',
 'rotSkyPos',
 'salIndex',
 'targetId', 'private_sndStamp']
dd = await efd_client.select_time_series(topic, fields, sunset, sunrise, index=2)
obs = dd
obs

print(f"On night {DAYOBS} aux tel recorded {len(obs)} observations") 

On night 2024-02-06 aux tel recorded 48 observations


In [11]:
# Make plot of acquired airmasses
visits_ac.hvplot(
    by=["note"],
    kind="scatter",
    x="start_date",
    ylim=(2.3, 0.9),
    y=["airmass"],
)


:NdOverlay   [note]
   :Scatter   [start_date]   (airmass)